# Web-application

## Introduction 
The a dash-application will be used to show-case the cluster collection after K-means clustering and RFM- analysis applied to the dataset.

## Purpose

The web-app helps users to filter through cluster infomation about clients, displaying useful information data-analysts can us to predict behavioural tendencies of 
an cluster's clients. The behavioural tendencies of the clusters are examined according to the frequency they shop from the enterprise and the total amount they spent at the company.

## Functionality

- User Interface: The web application works with a dropbox wich receives a processed CSV file, then tunes the cluster collection being previed by adjusting the values at the amount and frequency sliders of customers.
- Back-end programming: When droping a csv file into the dropbox, the dataset is preprocessed and a K-means clustering model is applied to the dataset and then the sliders adjusts the cluster collection the analyst using the web-application preceives, according to the amount and frequency.
- The output-display:  The display will display the RMF, k-means-clustered models and will adapt changes as the amount and frequency sliders are adjusted.

## Dash application functionality

The dash application is utilised to create interactive websites data-analysts can use to make model-predictions and showcase differences between cluster clients in our libraries. The website-application utilizes interactive packages with external CSS stylisation resources, HTML and Python scripts to make the dash-application design more intuative.

## Back-end functionality:

The following libraries from the source package will prepare the dataset and apply a K-means clustering model to the dataset to produce an dataset which can be visualised by the web-application.

## User interface design 

The following library will used to create a wep-application to display both the outlier K-means clustering model and the non-outlier K-means clustering model.